# Datenvorbereitung
Zunächst werden für alle gewünschten Einflussfaktoren die Daten beschafft und in ein CSV-Format mit lat/lon oder GeoJSON-Geoemtry gebracht, sodass zu allen Adressen die gewünschten Eigenschaften vorliegen.

In [ ]:
import pandas as pd
# Helper-Funktionen für Geodaten
from shapely.geometry import Point

EPSG_4326 = "EPSG:4326"

HAUSNUMMERZUSATZ = "HsnrZus"
HAUSNUMMER = "Hsnr"
STRASSENNAME = "Straßenname"

def load_geocsv(path, crs="EPSG:4326", geometry_col="geometry"):
    df = pd.read_csv(path, encoding="utf-8")

    # FALL 1: "geometry" existiert
    if geometry_col in df.columns:
        # Parse vorhandene Werte; falls leer, bleibt es None
        df[geometry_col] = df[geometry_col].apply(
            lambda x: wkt.loads(x) if isinstance(x, str) and x.startswith("POINT") else None
        )
    # FALL 2: "geometry" existiert NICHT
    else:
        if "lon" in df.columns and "lat" in df.columns:
            # Erzeuge komplett neue "geometry"-Spalte
            df[geometry_col] = df.apply(
                lambda row: Point(float(row["lon"]), float(row["lat"]))
                if pd.notna(row["lon"]) and pd.notna(row["lat"]) else None,
                axis=1
            )
        else:
            raise ValueError(
                "Fehlt sowohl 'geometry' als auch ('lat', 'lon')! "
                f"Gefunden: {df.columns.tolist()}"
            )
    # Mach GeoDataFrame
    gdf = gpd.GeoDataFrame(df, geometry=geometry_col, crs=crs)
    return gdf

def geo_sjoin(left, right, value_cols, how="left", predicate="intersects", drop_index_cols=True,
              suffixes=("", "_joined")):
    # Prüfe, ob Spalten bereits existieren
    already_present = [col for col in value_cols if col in left.columns]
    if already_present:
        print(f"Skip Join: Columns already present: {already_present}")
        return left
    # Nur Join, wenn noch nicht passiert
    result = gpd.sjoin(left, right[value_cols + ["geometry"]], how=how, predicate=predicate, lsuffix=suffixes[0],
                       rsuffix=suffixes[1])
    if drop_index_cols:
        result = result[[col for col in result.columns if not col.startswith("index_")]]
    result = result.reset_index(drop=True)
    return result

def make_merge_addr(row):
    s = str(row[STRASSENNAME]).strip().lower()
    hn = str(row[HAUSNUMMER]).strip().lower()
    hzusatz = str(row.get(HAUSNUMMERZUSATZ, '')).strip().lower() if HAUSNUMMERZUSATZ in row and not pd.isna(row.get(HAUSNUMMERZUSATZ, None)) else ""
    if hzusatz and hzusatz != "nan":
        hn += hzusatz
    adr = f"{s} {hn}".replace("  ", " ").strip()
    return adr

def add_markers_from_csv(
    map_obj,
    csv_path,
    color="blue",
    icon="info-sign",
    tooltip_format="{Straßenname} {Hsnr}{HsnrZus}",
    fallback_label="Unbekannte Adresse",
    layer_name=None
):
    """
    Fügt Marker aus einer CSV-Datei einer Folium-Karte hinzu.

    Parameter:
        map_obj (folium.Map): Die Karte, der Marker hinzugefügt werden.
        csv_path (str): Pfad zur CSV-Datei mit 'lat', 'lon', 'Straßenname', 'Hsnr', optional 'HsnrZus'.
        color (str): Marker-Farbe.
        icon (str): FontAwesome-Icon-Name.
        tooltip_format (str): Formatstring für Tooltip.
        fallback_label (str): Wird angezeigt, wenn kein sinnvoller Adressname gebildet werden kann.
        layer_name (str|None): Optionaler Layer-Name.
    """
    df = pd.read_csv(csv_path, encoding="utf-8")
    df.columns = [c.strip() for c in df.columns]
    df = df.dropna(subset=["lat", "lon"])

    layer = folium.FeatureGroup(name=layer_name) if layer_name else map_obj

    for _, row in df.iterrows():
        row_data = {k: (row.get(k, "") or "") for k in df.columns}

        # explizit fehlende Werte ersetzen
        strasse = str(row_data.get(STRASSENNAME, "")).strip()
        hsnr = str(row_data.get(HAUSNUMMER, "")).strip()
        hsnrzus = str(row_data.get(HAUSNUMMERZUSATZ, "")).strip()
        hat_adresse = any([strasse, hsnr, hsnrzus])

        tooltip = tooltip_format.format(Straßenname=strasse, Hsnr=hsnr, HsnrZus=hsnrzus) if hat_adresse else fallback_label

        marker = folium.Marker(
            location=[row["lat"], row["lon"]],
            icon=folium.Icon(color=color, icon=icon, prefix="fa", ),
            tooltip=tooltip
        )
        marker.add_to(layer)

    if layer_name:
        layer.add_to(map_obj)

def min_max(series, invert=False):
    s = series.copy()
    if invert:
        s = -s
    return (s - s.min()) / (s.max() - s.min())

## Adressen einlesen
Alle Adressen des Zielgebiets als CSV in einen GeoDataFrame einlesen.

In [ ]:
import geopandas as gpd
gdf_main = load_geocsv("data/adressen_mit_routen.csv")

## Ortsteile der Stadt visualisieren und im Datensatz ergänzen

In [ ]:
import json
import folium
import geopandas as gpd
from shapely.geometry import box

# Datei laden
with open("data/ortsteile_brandenburg.json", "r", encoding="utf-8") as f:
    raw = json.load(f)

# FeatureCollection extrahieren
features = raw["features"]
gdf_ortsteile = gpd.GeoDataFrame.from_features(features)
gdf_ortsteile.set_crs(EPSG_4326, inplace=True)

#  BBOX für Brandenburg an der Havel (nur relevante Ortsteile)
bbox = box(12.3120236786, 52.2938432979, 12.7562682548, 52.5594777244)
gdf_ortsteile = gdf_ortsteile.clip(bbox)

# Karte zentrieren
CITY_CENTER = (52.4116351153561, 12.556331280534392)
m = folium.Map(location=CITY_CENTER, zoom_start=12, tiles="cartodbpositron")

# Tooltip konfigurieren
tooltip_fields = ["otl_name"]
tooltip = folium.GeoJsonTooltip(
    fields=tooltip_fields,
    aliases=["Ortsteil"],
    localize=True,
    sticky=True
)

# GeoJSON mit Tooltip zur Karte hinzufügen
folium.GeoJson(
    gdf_ortsteile,
    name="Ortsteile",
    tooltip=tooltip
).add_to(m)

assert gdf_main.crs.to_epsg() == 4326
assert gdf_ortsteile.crs.to_epsg() == 4326

gdf_main = gpd.sjoin(gdf_main, gdf_ortsteile[["geometry", "otl_name"]], how="left", predicate="within")
gdf_main = gdf_main.rename(columns={"otl_name": "ortsteil"})
gdf_main = gdf_main.drop(columns=["index_right"], errors="ignore")

# Drop rows with NaN lat/lon
gdf_main = gdf_main.dropna(subset=["lat"])

# TODO Ergänzung einer Spalte "stadtteil" für spätere Validierung und auch Visualisierung
# Adressen im Zentrum haben korrekterweise keinen Ortsteil

# Karte anzeigen
m


## Explorative Datenanalyse

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns           # nur für schönere Plots
from helper import load_geocsv

# Eindeutigen Merge-baren String aus Adresse erzeugen
gdf_main["Adresse_merge"] = gdf_main.apply(make_merge_addr, axis=1)

gdf_main = gdf_main.rename(columns={"geojson": "center_route",
                                    "distance_m" : "center_distance"})

# Relevante Spalten auswählen
gdf_main = gdf_main[["Straßenname", "Hsnr", "HsnrZus",
         "center_distance", "lat", "lon", "geometry", "Adresse_merge", "ortsteil", "center_route"]]

# Histogramm der Distanzen zum Zentrum
plt.figure(figsize=(6,3))
sns.histplot(gdf_main["center_distance"], bins=60, kde=False)
plt.title("Verteilung der Fußentfernung zum Zentrum")
plt.xlabel("Distanz (Meter)")
plt.ylabel("Häufigkeit")
plt.show()
print(gdf_main.shape)
print(gdf_main.columns)

## Daten bereinigen

In [ ]:
print(gdf_main.shape)
# Dubletten aus Adressen entfernen
dups = gdf_main["Adresse_merge"].value_counts()
dups = dups[dups > 1]
print(f"{len(dups)} doppelte Adressen gefunden")
print(dups.head())

# Ersten Treffer behalten
gdf_main = gdf_main.sort_values("Adresse_merge").drop_duplicates("Adresse_merge", keep="first")
print(gdf_main.shape)


## Einzelhandel
Separates Skript ```einzelhandel-adreessen.py``` ausfühen, um Datei "adressen_mit_einzelhandel.csv" zu erzeugen.
Durch fußläufiges Routing berechnete Faktoren:
- Anzahl von Einkaufsmöglichkeiten im Umkreis von 500 m
- Anzahl von Einkaufsmöglichkeiten im Umkreis von 800 m
- Geringste Distanz zum nächsten Einzelhandel

In [ ]:
import seaborn as sns, matplotlib.pyplot as plt

# Lade vorberechnete Einzelhandel-Faktoren
gdf_retail = load_geocsv("data/adressen_mit_einzelhandel.csv")

# Merge mit Haupt-GDF
gdf_retail["Adresse_merge"] = gdf_retail.apply(make_merge_addr, axis=1)
gdf_retail = gdf_retail.sort_values("Adresse_merge").drop_duplicates("Adresse_merge", keep="first")

dups = gdf_retail["Adresse_merge"].value_counts()
print(dups[dups > 1])

einzelhandel_attribute = ["shop_min_m", "shops_500m_ct", "shops_800m_ct"]

gdf_main = pd.merge(
    gdf_main,
    gdf_retail[["Adresse_merge"] + einzelhandel_attribute],
    on="Adresse_merge",
    how="left",
    validate="one_to_many"
)
print(gdf_main.shape)
print(gdf_main[["Adresse_merge", "shop_min_m"]].head())

print(gdf_main.shape)

# Speicher freigeben
del(gdf_retail)

In [ ]:
# Verteilung Distanz zum nächsten Markt
print(gdf_main.columns)
sns.histplot(gdf_main["shop_min_m"].dropna(), bins=40, kde=False)
plt.title("Distanz zum nächsten Lebensmittel­markt")
plt.xlabel("Meter Fußweg"); plt.ylabel("Adressen")
plt.show()

# Scatter Zentralität vs. Nahversorgung
sns.scatterplot(x="center_distance", y="shop_min_m", data=gdf_main, alpha=.3)
plt.xlabel("Distanz Zentrum (m)"); plt.ylabel("Distanz nächster Markt (m)")
plt.show()

## Lärmbelastung
Siehe ```laerm.ipynb``` zur Erzeugung, ansonsten einfach ```data/adressen_mit_laerm.csv``` verwenden und per Spalte 'geometry' verschneiden.

Definition Lärm-Index:
- NaN / leer: kein gemessener Straßenlärm
- 0: 55 - 59 dB
- 1: 60 - 64 dB
- 2: 65 - 69 dB
- 3: 70 - 74 dB
- 4: >= 75 db

In [ ]:
gdf_laerm_karte = load_geocsv("data/adressen_mit_laerm.csv")
print(gdf_laerm_karte.shape)
print(gdf_main.shape)

# Merge des Lärmindex mit Haupt-GDF
gdf_laerm_karte["Adresse_merge"] = gdf_laerm_karte.apply(make_merge_addr, axis=1)
gdf_laerm_karte["Laerm_index_tag"] = gdf_laerm_karte["Laerm_index_tag"].fillna(-0.1)

# Deduplicate, im Zweifel nimm den lautesten Wert
gdf_laerm_karte = (
    gdf_laerm_karte
    .sort_values("Laerm_index_tag", ascending=False)
    .drop_duplicates("Adresse_merge", keep="first")
)

laerm_attribute = ["Laerm_index_tag"]
gdf_main = pd.merge(
    gdf_main,
    gdf_laerm_karte[["Adresse_merge"] + laerm_attribute],
    on="Adresse_merge",
    how="left",
    validate="one_to_many"
)

#print(gdf_main.columns)
print(gdf_main.shape)

## Bildung
Weitere Faktoren
- Fußläufige Entfernung zur nächstgelegenen Kita
- Fußläufige Entfernung zur nächstgelegenen Schule
- Anzahl Kitas im Umkreis von 500, 800 und 1000 Metern
- Anzahl Schulen im Umkreis von 500, 800 und 1000 Metern

In [ ]:
import pandas as pd
import geopandas as gpd
from shapely import wkt

# Daten als GeoDataFrame einlesen
gdf_kitas = load_geocsv("data/adressen_mit_kita_routen.csv")
gdf_grundschulen = load_geocsv("data/adressen_mit_grundschul_routen.csv")
print(gdf_kitas.shape)
print(gdf_grundschulen.shape)
print(gdf_main.shape)

# Eindeutige Adresse für den Merge generieren
gdf_kitas["Adresse_merge"] = gdf_kitas.apply(make_merge_addr, axis=1)
gdf_kitas = gdf_kitas.drop_duplicates("Adresse_merge")
gdf_grundschulen["Adresse_merge"] = gdf_grundschulen.apply(make_merge_addr, axis=1)
gdf_grundschulen = gdf_grundschulen.drop_duplicates("Adresse_merge")

kitas_attribute = ["kitas_min_distance_m", "kitas_geometry", "kitas_count_within_500m", "kitas_count_within_800m", "kitas_count_within_1000m"]
gdf_main = pd.merge(
    gdf_main,
    gdf_kitas[["Adresse_merge"] + kitas_attribute],
    on="Adresse_merge",
    how="left",
    validate="one_to_many"
)

grundschulen_attribute = ["grundschulen_min_distance_m", "grundschulen_geometry", "grundschulen_count_within_500m", "grundschulen_count_within_800m", "grundschulen_count_within_1000m"]
gdf_main = pd.merge(
    gdf_main,
    gdf_grundschulen[["Adresse_merge"] + grundschulen_attribute],
    on="Adresse_merge",
    how="left",
    validate="one_to_many"
)
del(gdf_kitas, gdf_grundschulen)  # Speicher freigeben

print(gdf_main.columns)
print(gdf_main.shape)

## ÖPNV-Qualität

Die Qualität des ÖPNV wird anhand der Fußläufigkeit zur nächsten Haltestelle und der Häufigkeit von Abfahrten (Headway) bewertet. Die Daten stammen vom Verkehrsverbund Berlin-Brandenburg (VBB), Lizenz: CC BY 4.0, [zu den Daten](https://unternehmen.vbb.de/digitale-services/datensaetze).

Vorausgesetzte Datensätze:
- ```data/GTFS/stops.txt```, der die Haltestellen mit ihren Geokoordinaten enthält.
- ```data/adressen_mit_haltestellen_routen.csv```, der die Routen zu den Haltestellen und Anzahl von Haltestellen im Radius 500m und 800m enthält.

Probleme mit Datenqualität:
- In der "2024_Haltestellen.csv" fehlen die Haltestellen "Libellenweg" und "Immenweg" (Linie B). Dadurch werden diese Haltestellen nicht berücksichtigt und Wege zur nächsten Haltestelle werden länger eingeschätzt.

In [ ]:
# Load GTFS stops data
stops = pd.read_csv("data/GTFS/stops.txt")
stops["geometry"] = stops.apply(lambda row: Point(row["stop_lon"], row["stop_lat"]), axis=1)
gdf_stops = gpd.GeoDataFrame(stops, geometry="geometry", crs=EPSG_4326)

# Convert to UTM for precise metric distance calculations
gdf_main = gdf_main.to_crs(epsg=32633)
gdf_stops = gdf_stops.to_crs(epsg=32633)

# Assign nearest stop to each address based on Point(X, Y) geometry
gdf_main["nearest_stop_id"] = gdf_main.geometry.apply(
    lambda pt: gdf_stops.loc[gdf_stops.distance(pt).idxmin(), "stop_id"]
)

# Read routes from adresses to stops and stop count from prepared CSV
gdf_haltestellen = load_geocsv("data/adressen_mit_haltestellen_routen.csv")
gdf_haltestellen["Adresse_merge"] = gdf_haltestellen.apply(make_merge_addr, axis=1)

# Merge haltestellen attributes into main GeoDataFrame
haltestellen_attribute = ["haltestellen_geometry", "haltestellen_min_distance_m", "haltestellen_count_within_500m", "haltestellen_count_within_800m"]
gdf_main = pd.merge(
    gdf_main,
    gdf_haltestellen[["Adresse_merge"] + haltestellen_attribute],
    on="Adresse_merge",
    how="left",
    validate="one_to_many"
)

# Speicher freigeben
del(gdf_haltestellen)

### Berechnung der ÖPNV-Taktung

In [ ]:
import pandas as pd

# 1. GTFS laden
stops = pd.read_csv("data/GTFS/stops.txt")
stop_times = pd.read_csv("data/GTFS/stop_times.txt", low_memory=False)
trips = pd.read_csv("data/GTFS/trips.txt")
calendar = pd.read_csv("data/GTFS/calendar.txt")

# 2. Zeit in Minuten umrechnen
def parse_time_to_minutes(t):
    try:
        h, m, s = map(int, t.split(":"))
        return h * 60 + m
    except:
        return None

stop_times["minutes"] = stop_times["arrival_time"].apply(parse_time_to_minutes)
stop_times = stop_times.dropna(subset=["minutes"])

# 3. Filter nur auf werktägliche Dienste
trips_filtered = trips.merge(calendar, on="service_id")
trips_filtered = trips_filtered[trips_filtered["monday"] == 1]  # oder beliebig anpassbar

# 4. Merge trips ↔ stop_times
stopdata = stop_times.merge(trips_filtered[["trip_id", "route_id"]], on="trip_id")

# 5. Headway-Funktion für beliebige Zeitfenster
def compute_headways(df, time_col="minutes", time_from=360, time_to=540):
    result = {}
    for stop_id, group in df.groupby("stop_id"):
        times = sorted(group[time_col])
        times = [t for t in times if time_from <= t <= time_to]
        if len(times) < 2:
            continue
        diffs = [b - a for a, b in zip(times, times[1:])]
        result[stop_id] = sum(diffs) / len(diffs)
    return result

# 6. Berechne morgens + abends
headway_morning = compute_headways(stopdata, time_from=360, time_to=540)     # 6–9 Uhr
headway_evening = compute_headways(stopdata, time_from=960, time_to=1140)   # 16–19 Uhr

# 7. In DataFrames umwandeln
df_hm = pd.DataFrame.from_dict(headway_morning, orient="index", columns=["headway_morning"]).reset_index().rename(columns={"index": "stop_id"})
df_he = pd.DataFrame.from_dict(headway_evening, orient="index", columns=["headway_evening"]).reset_index().rename(columns={"index": "stop_id"})
df_headways = df_hm.merge(df_he, on="stop_id", how="outer")

# 8. Merge mit gdf_main
gdf_main["nearest_stop_id"] = gdf_main["nearest_stop_id"].astype(str)
df_headways["stop_id"] = df_headways["stop_id"].astype(str)

# Headway-Spalten explizit aus gdf_main entfernen, falls sie existieren
for col in ["headway_morning", "headway_evening", "headway_avg", "stop_id"]:
    if col in gdf_main.columns:
        gdf_main.drop(columns=[col], inplace=True)

headway_attribute = ["headway_morning", "headway_evening"]
gdf_main = pd.merge(
    gdf_main,
    df_headways[["stop_id"] + headway_attribute],
    left_on="nearest_stop_id",
    right_on="stop_id",
    how="left",
    validate="many_to_one"
)

# Langfristig stabile Skala für Vergleichbarkeit
# Beispielwerte für "vernünftige" Bandbreite (z. B. Takt 5–60 Minuten)
fixed_min, fixed_max = 5, 60
for col in ["headway_morning", "headway_evening"]:
    score_col = col + "_score_fixed"
    gdf_main[score_col] = 1 - (
        (gdf_main[col] - fixed_min) / (fixed_max - fixed_min)
    ).clip(lower=0, upper=1)

# Durchschnittlicher Headway
gdf_main["headway_avg"] = gdf_main[["headway_morning", "headway_evening"]].mean(axis=1)

# Speicher freigeben
del(stops, stop_times, trips, calendar, df_hm, df_he, df_headways)

## Visualisierung der ÖPNV-Taktung

In [ ]:
import folium
from folium.plugins import MarkerCluster

# Basiskarte anlegen
m = folium.Map(location=CITY_CENTER, zoom_start=12, tiles="cartodbpositron")

# Farbskala vorbereiten (linear, abgestimmt auf deine Daten)
from branca.colormap import linear
vmin, vmax = gdf_main["headway_avg"].quantile([0.01, 0.99])  # Extremwerte abschneiden
colormap = linear.RdYlGn_11.scale(vmin, vmax).to_step(n=11)
colormap.caption = "Durchschnittlicher Headway (Minuten)"

# Adressen als Punkte (gefärbt nach Headway)
for idx, row in gdf_main.iterrows():
    if pd.notnull(row["headway_avg"]) and row.geometry:
        folium.CircleMarker(
            location=[row.geometry.y, row.geometry.x],
            radius=5,
            color=colormap(row["headway_avg"]),
            fill=True,
            fill_opacity=0.8,
            popup=f"Adresse: {row.get('Adresse_merge', idx)}<br>Headway: {row['headway_avg']:.1f} min"
        ).add_to(m)

# Haltestellen als schwarze Marker mit Cluster
marker_cluster = MarkerCluster(name="ÖPNV-Haltestellen").add_to(m)
for idx, row in gdf_stops.iterrows():
    if row.stop_lat and row.stop_lon:
        folium.CircleMarker(
            location=[row.stop_lon, row.stop_lat],
            radius=4,
            color="black",
            fill=True,
            fill_opacity=1,
            popup=row.get("stop_name", str(row.get("stop_id", idx)))
        ).add_to(marker_cluster)

# Legende
colormap.add_to(m)

# Layer control (optional)
folium.LayerControl().add_to(m)

# Karte anzeigen
m


# Visualisierung der Faktoren
Alle Einflussfaktoren (Supermärkte, Ärzte, Schulen etc.) werden zur visuellen Plausibilitätsprüfung auf einer Karte visualisiert.

In [ ]:
import folium
import branca.colormap as cm

TOOLTIP_FORMAT = "{Straßenname} {Hsnr}{HsnrZus}"
CITY_CENTER = (52.4116351153561, 12.556331280534392)
m = folium.Map(location=CITY_CENTER, zoom_start=13, tiles="cartodbpositron")

# Add layer markers
add_markers_from_csv(map_obj=m, csv_path="data/einzelhandel_geocoded.csv", color="blue", icon="shopping-cart", tooltip_format=(TOOLTIP_FORMAT), layer_name="Einzelhandel")
add_markers_from_csv( map_obj=m,csv_path="data/grundschulen_geocoded.csv",color="green",icon="graduation-cap",tooltip_format=TOOLTIP_FORMAT,layer_name="Grundschulen")
add_markers_from_csv(map_obj=m,csv_path="data/kitas_geocoded.csv", color="beige",icon="child",tooltip_format=TOOLTIP_FORMAT,layer_name="Kitas")
add_markers_from_csv(map_obj=m, csv_path="data/haltestellen_geocoded.csv",color="lightgray", icon="bus", tooltip_format=TOOLTIP_FORMAT,layer_name="Haltestellen")

# Lärmindex aus dem Geopackage
gdf_laerm_karte = gpd.read_file("data/laerm.gpkg")
bbox = box(12.3120236786, 52.2938432979, 12.7562682548, 52.5594777244)
gdf_laerm_karte = gdf_laerm_karte.clip(bbox)
value_column = "isov1"
min_val = gdf_laerm_karte[value_column].min()
max_val = gdf_laerm_karte[value_column].max()
colormap = cm.LinearColormap(colors=["green", "yellow", "red"], vmin=min_val, vmax=max_val)
colormap.caption = "Lärmpegel (LDEN in dB)"

# Alle Adressen als Punkte
adress_layer = folium.FeatureGroup(name="Wohnadressen")
for _, row in gdf_main.iterrows():
    if pd.notna(row["lat"]) and pd.notna(row["lon"]):
        strasse = str(row.get(HAUSNUMMER, "")).strip()
        hsnr = str(row.get(HAUSNUMMER, "")).strip()
        hsnrzus = str(row.get(HAUSNUMMERZUSATZ, "")).strip()

        tooltip = strasse + " " + hsnr
        
        adressen_map = folium.CircleMarker(
            location=[row["lat"], row["lon"]],
            radius=3,
            color="blue",
            fill=True,
            fill_opacity=0.6,
            tooltip=tooltip
        ).add_to(adress_layer)
adress_layer.add_to(m)

def style_function(feature):
    value = feature["properties"][value_column]
    return {
        "fillColor": colormap(value),
        "color": "black",
        "weight": 0.3,
        "fillOpacity": 0.2
    }

laerm_layer = folium.FeatureGroup(name="Lärmkarte (LDEN 2022)")
folium.GeoJson(
    gdf_laerm_karte,
    style_function=style_function,
).add_to(laerm_layer)
laerm_layer.add_to(m)
colormap.add_to(m)

# Schaltbare Layer
folium.LayerControl(collapsed=False).add_to(m)

del(gdf_laerm_karte)  # Speicher freigeben

m


## Faktoren
Hier werden z-Werte zu alle **Einflussfaktoren** gebildet, um die Abweichung einer Ausprägung vom Standard zu erfassen. Weiterhin werden die **Gewichte** festgelegt, mit denen die Faktoren in die Wohnlagenbewertung eingehen.

In [ ]:
from scipy.stats import zscore

gdf = gdf_main # zum vereinfachten Umgang

# Nur Zeilen mit vollständigen Daten verwenden
score_vars = (["center_distance"] +
              haltestellen_attribute +
              headway_attribute +
              einzelhandel_attribute +
              laerm_attribute +
              kitas_attribute +
              grundschulen_attribute
              )
mask_all = gdf[score_vars].notna().all(axis=1)

# Z‑Scores, fehlende Werte bleiben NaN
gdf.loc[mask_all, "z_centrality"]    = -zscore(gdf.loc[mask_all, "center_distance"])
gdf.loc[mask_all, "z_shop_distance"] = -zscore(gdf.loc[mask_all, "shop_min_m"])
gdf.loc[mask_all, "z_shop_near_500"] =  zscore(gdf.loc[mask_all, "shops_500m_ct"])
gdf.loc[mask_all, "z_shop_near_800"] =  zscore(gdf.loc[mask_all, "shops_800m_ct"])
gdf.loc[mask_all, "z_laerm_index_tag"] = -zscore(gdf.loc[mask_all, "Laerm_index_tag"])
gdf.loc[mask_all, "z_kita_distance"] = -zscore(gdf.loc[mask_all, "kitas_min_distance_m"])
gdf.loc[mask_all, "z_kita_near_500"] =  zscore(gdf.loc[mask_all, "kitas_count_within_500m"])
gdf.loc[mask_all, "z_kita_near_800"] =  zscore(gdf.loc[mask_all, "kitas_count_within_800m"])
gdf.loc[mask_all, "z_kita_near_1000"] =  zscore(gdf.loc[mask_all, "kitas_count_within_1000m"])
gdf.loc[mask_all, "z_grundschul_distance"] = -zscore(gdf.loc[mask_all, "grundschulen_min_distance_m"])
gdf.loc[mask_all, "z_grundschulen_near_500"] =  zscore(gdf.loc[mask_all, "grundschulen_count_within_500m"])
gdf.loc[mask_all, "z_grundschulen_near_800"] =  zscore(gdf.loc[mask_all, "grundschulen_count_within_800m"])
gdf.loc[mask_all, "z_grundschulen_near_1000"] =  zscore(gdf.loc[mask_all, "grundschulen_count_within_1000m"])
gdf.loc[mask_all, "z_haltestelle_distance"] = -zscore(gdf.loc[mask_all, "haltestellen_min_distance_m"])
gdf.loc[mask_all, "z_headway_score"] = -zscore(gdf.loc[mask_all, "headway_avg"])

# Score-Zusammenfassung nur bei vollständigen Daten
mm_central_score_vars = ["center_distance", "shop_min_m", "shops_500m_ct", "shops_800m_ct", "Laerm_index_tag"]
mask_mm_central = gdf[mm_central_score_vars].notna().all(axis=1)

gdf.loc[mask_mm_central, "score_central"] = (
    0.5 * gdf.loc[mask_mm_central, "center_distance"] +
    0.4 * (
        0.4 * gdf.loc[mask_mm_central, "shop_min_m"] +
        0.3 * gdf.loc[mask_mm_central, "shops_500m_ct"] +
        0.3 * gdf.loc[mask_mm_central, "shops_800m_ct"]
    ) +
    0.1 * gdf.loc[mask_mm_central, "Laerm_index_tag"]
)

# Kitas
# Anders als kita_attribute (kein "geometry", was bei der Maskierung leere Zeilen von geometry rausschmeißen würde)
mm_kita_score_vars = [
    "kitas_min_distance_m",
    "kitas_count_within_500m",
    "kitas_count_within_800m",
    "kitas_count_within_1000m"
]
mask_mm_kita = gdf[mm_kita_score_vars].notna().all(axis=1)
gdf.loc[mask_mm_kita, "score_kita"] = (
    0.5 * gdf.loc[mask_mm_kita, "kitas_min_distance_m"] +
    0.2 * gdf.loc[mask_mm_kita, "kitas_count_within_500m"] +
    0.2 * gdf.loc[mask_mm_kita, "kitas_count_within_800m"] +
    0.1 * gdf.loc[mask_mm_kita, "kitas_count_within_1000m"]
)

# Grundschulen
mm_grundschulen_score_vars = [
    "grundschulen_min_distance_m",
    "grundschulen_count_within_500m",
    "grundschulen_count_within_800m",
    "grundschulen_count_within_1000m"
]
mask_mm_grundschule = gdf[mm_grundschulen_score_vars].notna().all(axis=1)
gdf.loc[mask_mm_grundschule, "score_grundschule"] = (
    0.5 * gdf.loc[mask_mm_grundschule, "grundschulen_min_distance_m"] +
    0.2 * gdf.loc[mask_mm_grundschule, "grundschulen_count_within_500m"] +
    0.2 * gdf.loc[mask_mm_grundschule, "grundschulen_count_within_800m"] +
    0.1 * gdf.loc[mask_mm_grundschule, "grundschulen_count_within_1000m"]
)

# Gesamt-Score (falls alle Teil-Scores vorhanden)
score_all_vars = ["score_central", "score_kita", "score_grundschule", "z_haltestelle_distance", "z_headway_score"]
mask_all_scores = gdf[score_all_vars].notna().all(axis=1)

# Skalierte Kombination (je niedriger desto besser)
gdf.loc[mask_all_scores, "score_total"] = (
    0.4 * gdf.loc[mask_all_scores, "score_central"] +
    0.15 * gdf.loc[mask_all_scores, "score_kita"] +
    0.15 * gdf.loc[mask_all_scores, "score_grundschule"] +
    0.15 * gdf.loc[mask_all_scores, "z_haltestelle_distance"] +
    0.15 * gdf.loc[mask_all_scores, "z_headway_score"]
)

print("Anzahl gültiger Gesamt-Scores:", gdf["score_total"].notna().sum())
print(gdf[score_all_vars].notna().sum().sort_values())

all_input_vars = mm_central_score_vars + mm_kita_score_vars + mm_grundschulen_score_vars
missing_counts = gdf[all_input_vars].isna().sum().sort_values(ascending=False)
print("")
print(missing_counts)

In [ ]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

z_vars = [
    "z_centrality",
    "z_shop_distance", "z_shop_near_500", "z_shop_near_800",
    "z_laerm_index_tag",
    "z_kita_distance", "z_kita_near_500", "z_kita_near_800", "z_kita_near_1000",
    "z_grundschul_distance", "z_grundschulen_near_500", "z_grundschulen_near_800", "z_grundschulen_near_1000",
    "z_haltestelle_distance", "z_headway_score",
]
X = gdf[z_vars].dropna().values

inertia = []
cluster_range = range(2, 11)  # Du kannst bis 15 hochgehen

for k in cluster_range:
    model = KMeans(n_clusters=k, random_state=42)
    model.fit(X)
    inertia.append(model.inertia_)

plt.figure(figsize=(8,5))
plt.plot(cluster_range, inertia, marker='o')
plt.title("Elbow-Methode: KMeans-Inertia vs. Clusteranzahl")
plt.xlabel("Anzahl Cluster (k)")
plt.ylabel("Inertia (Distanz innerhalb der Cluster)")
plt.xticks(cluster_range)
plt.grid(True)


In [ ]:
# Silhouetten-Analyse
from sklearn.metrics import silhouette_score

silhouettes = []

for k in cluster_range:
    model = KMeans(n_clusters=k, random_state=42)
    labels = model.fit_predict(X)
    score = silhouette_score(X, labels, random_state=42)
    silhouettes.append(score)

plt.figure(figsize=(8,5))
plt.plot(cluster_range, silhouettes, marker='o', color="green")
plt.title("Silhouetten-Score pro Clusteranzahl")
plt.xlabel("Anzahl Cluster (k)")
plt.ylabel("Durchschn. Silhouetten-Koeffizient")
plt.xticks(cluster_range)
plt.grid(True)
plt.show()


In [ ]:
from sklearn.cluster import KMeans

X = gdf[z_vars].dropna().values  # Nur vollständige Zeilen

model = KMeans(n_clusters=5, random_state=42).fit(X)

mask_complete = gdf[z_vars].notna().all(axis=1)
gdf.loc[mask_complete, "cluster"] = model.labels_

cluster_centers = pd.DataFrame(model.cluster_centers_, columns=z_vars)
cluster_centers.index.name = "Cluster"
print(cluster_centers.round(2))


# Visualisierung mit Dimension Reduction
from sklearn.decomposition import PCA
X_scaled = X  # bereits Z-Scores → keine erneute Skalierung nötig
pca = PCA(n_components=2, random_state=42)
X_pca = pca.fit_transform(X_scaled)

plt.figure(figsize=(8,6))
plt.scatter(X_pca[:,0], X_pca[:,1], c=model.labels_, cmap="tab10", alpha=0.6)
plt.title("KMeans-Cluster (PCA 2D-Projektion)")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.show()

In [ ]:
import pandas as pd, folium

gdf = gdf[gdf["lat"].notna() & gdf["lon"].notna() & gdf["cluster"].notna()]
gdf["cluster"] = gdf["cluster"].astype(int)

# Farbpalette für 5 Cluster
cluster_colors = {
    0: "#e41a1c",   # Cluster 0 - rot
    1: "#377eb8",   # Cluster 1 - blau
    2: "#4daf4a",   # Cluster 2 - grün
    3: "#984ea3",   # Cluster 3 - lila
    4: "#ff7f00",   # Cluster 4 - orange
    5: "#666666",
    6: "#a65628",
    7: "#66cd2c",
    # Füge weitere hinzu falls nötig!
}

# Farben für Wohnlagen (grün - gelb)
color_map = {
    "1 Top":      "#fee08b",
    "2":          "#d9ef8b",
    "3":          "#a6d96a",
    "4":          "#66bd63",
    "5 schwach":  "#1a9850",
}

m = folium.Map(location=CITY_CENTER, zoom_start=13, tiles="cartodbpositron")

# Add layer markers
add_markers_from_csv(map_obj=m, csv_path="data/einzelhandel_geocoded.csv", color="blue", icon="shopping-cart", tooltip_format=(TOOLTIP_FORMAT), layer_name="Einzelhandel")
add_markers_from_csv( map_obj=m,csv_path="data/grundschulen_geocoded.csv",color="green",icon="graduation-cap",tooltip_format=TOOLTIP_FORMAT,layer_name="Grundschulen")
add_markers_from_csv(map_obj=m,csv_path="data/kitas_geocoded.csv", color="beige",icon="child",tooltip_format=TOOLTIP_FORMAT,layer_name="Kitas")
#add_markers_from_csv(map_obj=m,csv_path="data/haltestellen_geocoded.csv",color="lightgray", icon="bus", tooltip_format=TOOLTIP_FORMAT,layer_name="Haltestellen")

valid_kita_json = gdf_main["kitas_geometry"].apply(lambda x: isinstance(x, str))
print("Gültige JSON-Einträge:", valid_kita_json.sum(), "/", len(gdf_main))

for _, r in gdf.iterrows():
    c = cluster_colors.get(r.cluster, "#666666")
    folium.CircleMarker(
        location=[r.lat, r.lon],
        radius=3,
        color=c,
        fill=True,
        fill_color=c,
        fill_opacity=0.85,
        tooltip=f"{r.Straßenname} {r.Hsnr} – Cluster {r.cluster}"
    ).add_to(m)

# Optional: Layer Control
folium.LayerControl().add_to(m)

print(gdf.shape)

#m.save("wohnlagen_clusterkarte.html")
m

# Validierung

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

zscore_cols = [
    "z_centrality",
    "z_shop_distance",
    "z_laerm_index_tag",
    "z_kita_distance",
    "z_grundschul_distance",
    "z_haltestelle_distance",
    "z_headway_score",
    "z_haltestellen_count_500"
]

# Nur Spalten verwenden, die im gdf enthalten sind
cols = [col for col in zscore_cols if col in gdf.columns]

# Korrelationsmatrix berechnen
corr = gdf[cols].corr()

# Plot
plt.figure(figsize=(10, 8))
sns.heatmap(corr, annot=True, cmap="vlag", center=0)
plt.title("Korrelationsmatrix der Z-Scores")
plt.tight_layout()
plt.show()


## Interaktive Karte für Bewertung einzelner Adressen

In [ ]:
import folium
import json
import ipywidgets as widgets
from IPython.display import display, clear_output

# Widget für Adresseingabe
text_input = widgets.Text(
    value='Immenweg 56',
    placeholder='Straßenname Hausnummer',
    description='Adresse:',
    disabled=False
)

# Button
button = widgets.Button(description="Zeige Routen")

# Ausgabe-Bereich für die Karte
output = widgets.Output()

# Funktion zum Einfügen einer Route + Zielmarker
def add_route(m, geojson_str, color, label, icon, distance=None):
    if isinstance(geojson_str, str):
        try:
            geo = json.loads(geojson_str)
            if geo.get("type") == "LineString":
                coords = [(y, x) for x, y in geo["coordinates"]]
                distance_text = f" – {int(distance)} m" if distance else ""
                tooltip_text = f"{label}{distance_text}"

                folium.PolyLine(
                    locations=coords,
                    color=color,
                    weight=4,
                    opacity=0.9,
                    tooltip=tooltip_text
                ).add_to(m)

                end = coords[-1]
                folium.Marker(
                    location=end,
                    icon=folium.Icon(color=color, icon=icon, prefix="fa"),
                    tooltip=f"Ziel: {label}{distance_text}"
                ).add_to(m)
        except Exception as e:
            print(f"Fehler bei {label}: {e}")

# Funktion zum Anzeigen der Karte
def show_routes(b):
    with output:
        clear_output()
        addr = text_input.value.strip().lower()

        filtered = gdf_main[gdf_main["Adresse_merge"].str.lower().str.contains(addr)]
        if filtered.empty:
            print("Keine Adresse gefunden.")
            return

        row = filtered.iloc[0]
        m = folium.Map(location=[row.lat, row.lon], zoom_start=15, tiles="cartodbpositron")
        folium.Marker(location=[row.lat, row.lon], tooltip="Adresse").add_to(m)

        # ▸ Routen einfügen
        add_route(m, row.get("kitas_geometry"), "orange", "Nächste Kita", "child", row.get("kitas_min_distance_m"))
        add_route(m, row.get("grundschulen_geometry"), "green", "Nächste Grundschule", "graduation-cap", row.get("grundschulen_min_distance_m"))
        add_route(m, row.get("center_route"), "lightgray", "Weg zum Zentrum", "arrows-to-circle", row.get("center_distance"))
        add_route(m, row.get("haltestellen_geometry"), "gray", "Nächste Haltestelle", "bus", row.get("haltestellen_min_distance_m"))

        display(m)

# Button-Event
button.on_click(show_routes)

# UI anzeigen
display(text_input, button, output)